In [1]:
import json
from collections import defaultdict

from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, clear_output
import ipywidgets as widgets
import plotly.graph_objs as go
import plotly
import numpy as np
import numpy.linalg as nla

import sim_lib.util as util
import sim_lib.graph_vis as graph_vis

init_notebook_mode(connected=True)

In [2]:
_SIMPLEST = False

filename = 'ntwk_data/ntwk_data.json'
if _SIMPLEST:
    filename = 'ntwk_data/ntwk_data_simplest.json'

In [3]:
ntwk_data = {}
with open(filename, 'r') as ntd:
    ntwk_data = json.loads(ntd.read())

In [4]:
class NtwkGraphVis:
    def __init__(self, data, radius):
        
        self.data = data
        self.radius = radius
        self.cur_run = 0

        r_vals = list(self.data.keys())
        p_vals = list(self.data[r_vals[0]].keys())
        
        self.p_val = widgets.Select(
                                options=p_vals,
                                description='Transmission probability',
                                disabled=False)
        self.r_val = widgets.Select(
                                options=r_vals,
                                description='Resources per vertex',
                                disabled=False)

        self.ntwk_types = widgets.SelectMultiple(
                                    options=['rl', 'ws', 'er', 'cm', 'kg'],
                                    description='Network type',
                                    disabled=False)
        
        # Update button
        self.plot_button = widgets.Button(
                                    description='Plot',
                                    disabled=False,
                                    button_style='',
                                    tooltip='Plots by selected parameters')
        self.plot_button.on_click(self.plot)
        
        # Next run button
        self.next_run_button = widgets.Button(
                                    description='Show the next run',
                                    disabled=False,
                                    button_style='',
                                    tooltip='Plots the next run of the simulation')
        self.next_run_button.on_click(self.next_run)
        self.run_counter_header =  widgets.HTML(
                            value=f'Run: {self.cur_run}',
                            placeholder='Some HTML',
                            description='')
        
        self.visuals = [self.p_val, self.r_val, self.ntwk_types, self.plot_button,
                        self.next_run_button, self.run_counter_header]
        
        self.render_visuals()
        
    def render_visuals(self):

        # Render widgets
        for vis in self.visuals:
            display(vis)
            
    def next_run(self, button):
        self.cur_run += 1
        
        p_val = self.p_val.value
        r_val = self.r_val.value
        ntwk_types = self.ntwk_types.value
        
        for ntwk in ntwk_types:
            if self.cur_run > len(self.data[r_val][p_val]['utils'][ntwk]):
                self.cur_run = 0
                break
                
        self.run_counter_header.value = f'Run: {self.cur_run}'
        self.plot_visuals()

    def plot(self, button):
        self.plot_visuals()
            
    def plot_visuals(self):
        # Reset widgets
        clear_output()
        self.render_visuals()
        
        p_val = self.p_val.value
        r_val = self.r_val.value
        
        ntwk_types = self.ntwk_types.value
            
        for ntwk in ntwk_types:
            sim_util = self.data[r_val][p_val]['utils'][ntwk][self.cur_run]
            sim_graph_ser = self.data[r_val][p_val]['graphs'][ntwk][self.cur_run]
            sim_graph = util.json_to_graph(sim_graph_ser)
            
            if ntwk == 'kg':
                g_fig = graph_vis.vis_G(sim_graph, self.radius, sim_util, mode='grid', ntwk_name=ntwk)
            else:
                g_fig = graph_vis.vis_G(sim_graph, self.radius, sim_util, mode='ring', ntwk_name=ntwk)

            iplot(g_fig)

In [5]:
ngv = NtwkGraphVis(ntwk_data, 10)

Select(description='Transmission probability', index=8, options=('1.0', '0.9', '0.8', '0.7', '0.6', '0.5', '0.…

Select(description='Resources per vertex', index=4, options=('128', '64', '32', '16', '8', '4', '2'), value='8…

SelectMultiple(description='Network type', index=(0, 1, 2, 3, 4), options=('rl', 'ws', 'er', 'cm', 'kg'), valu…

Button(description='Plot', style=ButtonStyle(), tooltip='Plots by selected parameters')

Button(description='Show the next run', style=ButtonStyle(), tooltip='Plots the next run of the simulation')

HTML(value='Run: 0', placeholder='Some HTML')

TypeError: float() argument must be a string or a number, not 'list'